# 第12章 判别分析及Python算法

## 12.1 判别分析的概念

In [ ]:
%run init.py

### 12.1.1 判别分析的用途

### 12.1.2 判别分析的种类

## 12.2 Fisher判别法

### 12.2.1 Fisher判别的提出

### 12.2.2 线性判别函数

In [ ]:
#%run init.py

In [ ]:
import pandas as pd                     
pd.set_option('display.precision',4)
pd.set_option('display.max_rows',20)       

In [ ]:
d121=pd.read_excel('mvsData.xlsx','d121');print(d121)

In [ ]:
d121.boxplot(column=['x1','x2'],by='G');

In [ ]:
#G1=d121[d121.G==1];G2=d121[d121.G==2];

In [ ]:
from scipy import stats as st
#st.levene(G1.x1,G2.x1)#方差齐性检验

In [ ]:
t1=st.ttest_ind(d121[d121.G==1].x1,d121[d121.G==2].x1)  #x1两类t检验
print('变量x1的两类t检验：t=%.4f, p=%.4f'%(t1[0],t1[1]))

In [ ]:
t2=st.ttest_ind(d121[d121.G==1].x2,d121[d121.G==2].x2)  #x2两类t检验
print('变量x2的两类t检验：t=%.4f, p=%.4f'%(t2[0],t2[1]))

In [ ]:
#import statsmodels.api as sm
#Logit=sm.GLM.from_formula("C(G)~x1+x2",d121,family=sm.families.Binomial()).fit()
#print(Logit.summary().tables[1])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(d121.x1,d121.x2,'.');plt.xlabel('x1');plt.ylabel('x2')
for i in range(0,len(d121.G)):plt.text(d121.x1[i],d121.x2[i],d121.G[i])

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda #?lda

In [ ]:
X=d121[['x1','x2']]; Y=d121['G']
d121_lda=lda(priors=(1/2,1/2)).fit(X,Y)
d121_lda

In [ ]:
d121['ld_G'] = d121_lda.predict(X) #线性判别结果
print(d121)

In [ ]:
#tab1=pd.crosstab(d121.G,d121.ld_G,margins=True)  
tab1=d121.pivot_table('id','G','ld_G',aggfunc=len,margins=True)
print('混淆矩阵:\n',tab1)

In [ ]:
import numpy as np
def Rate(tab): #定义计算符合率函数    
    rate=sum(np.diag(tab)[:-1]/np.diag(tab)[-1:])*100
    print('符合率: %.2f'%rate)     
Rate(tab1)

In [ ]:
d121_lda.scalings_  #判别系数

In [ ]:
a1=d121_lda.scalings_[0];a2=d121_lda.scalings_[1]
#y=a1x1+a2x2 ---> x2=1/a2(y-a1x1)  #线性判别方程
a=1/a2*(a1*np.mean(d121.x1)+a2*np.mean(d121.x2)); b =-a1/a2 
plt.plot(d121.x1,d121.x2,'.');plt.xlabel('x1');plt.ylabel('x2') 
plt.plot(d121.x1,a+b*d121.x1,color='r');#画出判别线
for i in range(0,len(d121.G)):plt.text(d121.x1[i],d121.x2[i],d121.G[i])

In [ ]:
print('线性判别分类情况：',d121_lda.predict(pd.DataFrame([[8.1,2.0]]))) 

### 12.2.3 非线性判别函数

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
d121_qda=qda(priors=(1/2,1/2)).fit(X,Y)
d121_qda

In [ ]:
d121['qd_G'] = d121_qda.predict(X)  #二次判别结果
print(d121)

In [ ]:
#tab2=pd.crosstab(d121.G,d121.qd_G,margins=True);
tab2=d121.pivot_table('id','G','qd_G',aggfunc=len,margins=True)
print('混淆矩阵:\n',tab2)

In [ ]:
Rate(tab2) 

In [ ]:
print('二次判别分类情况：',d121_qda.predict(pd.DataFrame([[8.1,2.0]]))) 

### 12.2.4 多总体距离判别

In [ ]:
d123=pd.read_excel('mvsData.xlsx','d123');#d123

In [ ]:
plt.plot(d123.Q,d123.C,'.'); plt.xlabel('Q');plt.ylabel('C')
for i in range(0,len(d123.G)): plt.text(d123.Q[i],d123.C[i],d123.G[i])

In [ ]:
plt.plot(d123.Q,d123.P,'.'); plt.xlabel('Q');plt.ylabel('P')
for i in range(0,len(d123.G)):plt.text(d123.Q[i],d123.P[i],d123.G[i])

In [ ]:
plt.plot(d123.C,d123.P,'.'); plt.xlabel('C');plt.ylabel('P')
for i in range(0,len(d123.G)):plt.text(d123.C[i],d123.P[i],d123.G[i])

In [ ]:
X=d123[['Q','C','P']]; Y=d123.G
d123_lda=lda(priors=(1/3,1/3,1/3)).fit(X,Y) #等概率
d123['ld_G'] = d123_lda.predict(X) #线性判别结果
print(d123)

In [ ]:
#d123_M1=pd.crosstab(d123.G,d123.ld_G,margins=True); 
M1=d123.pivot_table('id','G','ld_G',aggfunc=len,margins=True,fill_value='0')
print('混淆矩阵:\n',M1)

In [ ]:
Rate(M1)

In [ ]:
X0=pd.DataFrame([[8,7.5,65]])
print('线性判别分类情况：',d123_lda.predict(X0)) 

In [ ]:
d123_qda=qda(priors=(1/3,1/3,1/3)).fit(X,Y)
d123['qd_G'] = d123_qda.predict(X) #非线性判别结果
print(d123)

In [ ]:
M2=d123.pivot_table('id','G','qd_G',aggfunc=len,margins=True,fill_value='0')
print('混淆矩阵:\n',M2)

In [ ]:
Rate(M2)

In [ ]:
print('非线性判别分类情况：',d123_qda.predict(X0)) 

## 12.3 Bayes判别法

### 12.3.1 Bayes判别准则

### 12.3.2 正态总体的Bayes判别

In [ ]:
#先验概率不等的Bayes线性判别模型
d123_ld_B=lda(priors=(5/20,8/20,7/20)).fit(X,Y)
d123['ld_B_G'] = d123_ld_B.predict(X) 
print(d123)

In [ ]:
M3=d123.pivot_table('id','G','ld_B_G',aggfunc=len,margins=True,fill_value='0')
print('混淆矩阵:\n',M3)

In [ ]:
Rate(M3)

In [ ]:
print('分类后验概率:')
d123_ld_B_p=100*d123_ld_B.predict_proba(X).round(2)
print(pd.DataFrame(d123_ld_B_p,d123.id,['G1','G2','G3']))

In [ ]:
print('线性判别分类情况：',d123_ld_B.predict(X0))

In [ ]:
np.set_printoptions(precision=2,suppress=True) 
print('线性判别分类概率：',d123_ld_B.predict_proba(X0)*100)

In [ ]:
#先验概率不等的Bayes非线性判别模型
d123_qd_B=qda(priors=(5/20,8/20,7/20)).fit(X,Y)
d123['qd_B_G'] = d123_qd_B.predict(X) 
print(d123)

In [ ]:
M4=d123.pivot_table('id','G','qd_B_G',aggfunc=len,margins=True,fill_value='0')
print('混淆矩阵:\n',M4)

In [ ]:
Rate(M4)

In [ ]:
print('分类后验概率:')
d123_qd_B_p=100*d123_qd_B.predict_proba(X).round(2)
print(pd.DataFrame(d123_qd_B_p,d123.id,['G1','G2','G3']))

In [ ]:
print('非线性判别分类情况：',d123_qd_B.predict(X0))

In [ ]:
print('非线性判别分类概率：',d123_qd_B.predict_proba(X0)*100)

## 案例十二

In [ ]:
Case12=pd.read_excel('mvsCase.xlsx','Case12');Case12

In [ ]:
Case12.pivot_table(['CF_TD','NI_TA','CA_CL','CA_NS'],['G'],
                   aggfunc={len,np.mean,np.std})

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
X=Case12[['CF_TD','NI_TA','CA_CL','CA_NS']]; Y=Case12.G 
Case12_lda=lda().fit(X,Y)  #线性判别
Case12['ld_G'] = Case12_lda.predict(X)
Case12

In [ ]:
Case12_ld_M=Case12.pivot_table('id','G','ld_G',aggfunc=len,margins=True)
print('混淆矩阵:\n',Case12_ld_M)

In [ ]:
Rate(Case12_ld_M)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda
Case12_qda=qda().fit(X,Y)  #非线性判别
Case12['qd_G'] = Case12_qda.predict(X)
Case12

In [ ]:
Case12_qd_M=Case12.pivot_table('id','G','qd_G',aggfunc=len,margins=True)
print(Case12_qd_M)

In [ ]:
Rate(Case12_qd_M)